# Scraper
-----
- Load most recent weather.csv
- Collect list of missing data between start_date and today
- Scrape balloon data for any missing dates
- Clean / normalize new data
- Concatenate new and histrolcal DataFrames
- Update weather.csv
---

In [292]:
import os
import requests
import pandas as pd
from datetime import date, datetime, timedelta
from bs4 import BeautifulSoup
import numpy as np
from math import floor
from pathlib import Path  
from alive_progress import alive_bar
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import seaborn as sns
from windrose import WindroseAxes
from PIL import Image

## Load Historical Weather Data
---

In [293]:
data_path = './data/weather.csv'

# If data exists, load it
# ~~~~~~~~~~~~~~~~~~~~~~~
if os.path.exists(data_path):
    weather_data = pd.read_csv(data_path)
    print(weather_data.head())
else:
    print("No historical data exists")

         Date     Station  Pressure  Height  Temp  DewPoint  \
0  2020-01-01  Quillayute       700  2969.0  -0.9      -0.9   
1  2020-01-01  Quillayute       850  1393.0   6.2       6.2   
2  2020-01-01      Vernon       700  2880.0  -5.5      -7.0   
3  2020-01-01      Vernon       850  1357.0  -3.9      -5.8   
4  2020-01-01  Port Hardy       700  2856.0  -4.3      -5.8   

   Relative_Humidity  Mean_Mixed_Layer  Wind_Direction  Wind_Speed  \
0              100.0              5.14           255.0     100.008   
1              100.0              7.04           260.0      81.488   
2               89.0              3.25           260.0      94.452   
3               87.0              2.93           170.0      18.520   
4               89.0              3.56           250.0      87.044   

   Potential_Temp  Equivalent_Potential_Temp  Virtual_Potential_Temp  
0           301.5                      317.2                   302.4  
1           292.6                      313.1              

## Create Range of Missing Dates
---
Extension: Check any dates that are incomplete
- Missing one location but not all
- Missing values

In [294]:
dates = []
delta = timedelta(days=1)
start_date = date(2020, 1, 1)
end_date = datetime.today().date()

# Create list of all dates between start_date and today
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
while start_date <= end_date:
    # add current date to list by converting  it to iso format
    dates.append(start_date.isoformat())
    # increment start date by timedelta
    start_date += delta

In [295]:
# Display start and end of date range
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
for x in range(5):
    print(dates[x])
print('...')   
for x in range(5):
    print(dates[-x])

2020-01-01
2020-01-02
2020-01-03
2020-01-04
2020-01-05
...
2020-01-01
2023-06-28
2023-06-27
2023-06-26
2023-06-25


In [296]:
removed = []
# Remove dates from list that are alrady populated in weather_data
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
for d in weather_data['Date'].unique().tolist():
    if d in dates:
        dates.remove(d)
        removed.append(d)

### Extension
would it also be worth checking for entries with missing values?

In [297]:
# Display start and end of date range
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
for d in removed:
    print(d)

2020-01-01
2020-01-02
2020-01-03
2020-01-04
2020-01-05
2020-01-06
2020-01-07
2020-01-08
2020-01-09
2020-01-10
2020-01-11
2020-01-12
2020-01-13
2020-01-14
2020-01-15
2020-01-16
2020-01-17
2020-01-18
2020-01-19
2020-01-20
2020-01-21
2020-01-22
2020-01-23
2020-01-24
2020-01-25
2020-01-26
2020-01-27
2020-01-28
2020-01-29
2020-01-30
2020-01-31
2020-02-01
2020-02-02
2020-02-03
2020-02-04
2020-02-05
2020-02-06
2020-02-07
2020-02-08
2020-02-09
2020-02-10
2020-02-11
2020-02-12
2020-02-13
2020-02-14
2020-02-15
2020-02-16
2020-02-17
2020-02-18
2020-02-19
2020-02-20
2020-02-21
2020-02-22
2020-02-23
2020-02-24
2020-02-25
2020-02-26
2020-02-27
2020-02-28
2020-02-29
2020-03-01
2020-03-02
2020-03-03
2020-03-04
2020-03-05
2020-03-06
2020-03-07
2020-03-08
2020-03-09
2020-03-10
2020-03-11
2020-03-12
2020-03-13
2020-03-14
2020-03-15
2020-03-16
2020-03-17
2020-03-18
2020-03-19
2020-03-20
2020-03-21
2020-03-22
2020-03-23
2020-03-24
2020-03-25
2020-03-26
2020-03-27
2020-03-28
2020-03-29
2020-03-30
2020-03-31

## Generate URLs to be Scraped
---

In [298]:
base_url = "http://weather.uwyo.edu/cgi-bin/sounding?region=naconf&TYPE=TEXT%3ALIST"

stations = [
    {"id": 72797, "name": "Quillayute"},
    {"id": 73033, "name": "Vernon"},
    {"id": 71109, "name": "Port Hardy"}]

In [299]:
for d in dates:
    print(d)

2023-06-22
2023-06-23
2023-06-24
2023-06-25
2023-06-26
2023-06-27
2023-06-28


In [300]:
# Create list of URLs for each station
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
urls = []
dates_idx = []
station_idx = []
for d in dates:
    d = d.split('-')
    for station in stations:
        url = base_url
        url += "&YEAR={}".format(d[0])
        url += "&MONTH={}".format(d[1])
        #url += "&FROM=" + str(now[1]) + str(now[2])
        #url += "&TO=" + str(now[1]) + str(now[2])
        url += "&FROM={:0>2d}00".format(int(d[2]))
        url += "&TO={:0>2d}23".format(int(d[2]))
        url += "&STNM={}".format(station["id"])
        url += "&REPLOT=1"
        urls.append(url)
        
        
        dates_idx.append("%02d-%02d-%02d" % (int(d[0]), int(d[1]), int(d[2])))
        station_idx.append(station["name"])


In [301]:
for url in urls:
    print(url)

http://weather.uwyo.edu/cgi-bin/sounding?region=naconf&TYPE=TEXT%3ALIST&YEAR=2023&MONTH=06&FROM=2200&TO=2223&STNM=72797&REPLOT=1
http://weather.uwyo.edu/cgi-bin/sounding?region=naconf&TYPE=TEXT%3ALIST&YEAR=2023&MONTH=06&FROM=2200&TO=2223&STNM=73033&REPLOT=1
http://weather.uwyo.edu/cgi-bin/sounding?region=naconf&TYPE=TEXT%3ALIST&YEAR=2023&MONTH=06&FROM=2200&TO=2223&STNM=71109&REPLOT=1
http://weather.uwyo.edu/cgi-bin/sounding?region=naconf&TYPE=TEXT%3ALIST&YEAR=2023&MONTH=06&FROM=2300&TO=2323&STNM=72797&REPLOT=1
http://weather.uwyo.edu/cgi-bin/sounding?region=naconf&TYPE=TEXT%3ALIST&YEAR=2023&MONTH=06&FROM=2300&TO=2323&STNM=73033&REPLOT=1
http://weather.uwyo.edu/cgi-bin/sounding?region=naconf&TYPE=TEXT%3ALIST&YEAR=2023&MONTH=06&FROM=2300&TO=2323&STNM=71109&REPLOT=1
http://weather.uwyo.edu/cgi-bin/sounding?region=naconf&TYPE=TEXT%3ALIST&YEAR=2023&MONTH=06&FROM=2400&TO=2423&STNM=72797&REPLOT=1
http://weather.uwyo.edu/cgi-bin/sounding?region=naconf&TYPE=TEXT%3ALIST&YEAR=2023&MONTH=06&FROM=2

## Extract data from list of URLs
---
- Iterate through URLs
- Extract 700 and 850 data items
- Append data to array

In [302]:
data = []
reading_idx = []
problem_urls = []

print("Extracting data from URLs: ")
with alive_bar(len(urls), force_tty=True) as bar:
    for i, url in enumerate(urls):
        search = 0
        while search >= 0:
            # Scrape site data
            # ~~~~~~~~~~~~~~~~
            search += 1
            page = requests.get(url)
            soup = BeautifulSoup(page.content, "html.parser")
            results = str(soup.find('pre'))
            sevhundy = results[results.find("700"):].split()[:11]
            eightfiddy = results[results.find("850"):].split()[:11]
            
            # Catch erroneous reads
            # Site can hit too many requests
            # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            if 'e' not in sevhundy[0] and 'e' not in eightfiddy[0]:
                search = -1
            elif search == 10:
                print(url)
                problem_urls.append(url)
                search = -1

        # Extract 700 first // 850 second
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        data.append(sevhundy)
        data[-1].insert(0, dates_idx[i])
        data[-1].insert(1, station_idx[i])
        
        data.append(eightfiddy)
        data[-1].insert(0, dates_idx[i])
        data[-1].insert(1, station_idx[i])
        
        bar()
        

print("\nComplete")

Extracting data from URLs: 
|████████████████████████████████████████| 21/21 [100%] in 47.2s (0.44/s)                                               

Complete


In [303]:
for row in data:
    print(row)

['2023-06-22', 'Quillayute', '700.0', '3130', '-1.9', '-6.9', '69', '3.27', '15', '15', '300.4', '310.5', '300.9']
['2023-06-22', 'Quillayute', '850.0', '1551', '8.6', '-1.4', '49', '4.08', '320', '8', '295.1', '307.4', '295.9']
['2023-06-22', 'Vernon', '700.0', '3108', '-3.5', '-5.7', '85', '3.59', '275', '7', '298.6', '309.6', '299.2']
['2023-06-22', 'Vernon', '850.0', '1539', '8.8', '-0.2', '53', '4.46', '235', '6', '295.4', '308.7', '296.1']
['2023-06-22', 'Port Hardy', '700.0', '3127', '-1.1', '-16.1', '31', '1.56', '325', '8', '301.2', '306.3', '301.5']
['2023-06-22', 'Port Hardy', '850.0', '1550', '9.4', '-2.6', '43', '3.73', '295', '8', '296.0', '307.3', '296.6']
['2023-06-23', 'Quillayute', '700.0', '3088', '0.6', '-12.4', '37', '2.12', '55', '10', '303.1', '309.9', '303.5']
['2023-06-23', 'Quillayute', '850.0', '1501', '11.4', '-1.6', '40', '4.02', '80', '6', '298.1', '310.4', '298.8']
['2023-06-23', 'Vernon', '700.0', '3103', '-1.1', '-7.1', '64', '3.22', '165', '6', '301.2'

In [304]:
print("Problem URLs:")
for url in problem_urls:
    print(url)

Problem URLs:


## Clean the Extracted Data
---

In [305]:
# Remove rogue "e" values from pressure field
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
pressures = [700, 850]
ctr = 0
for item in data:
    if(not str(item[2]).isnumeric()):
        item[2] = pressures[ctr%2]
    ctr += 1

In [306]:
cols = ["Date", 
        "Station", 
        "Pressure", 
        "Height", 
        "Temp", 
        "DewPoint", 
        "Relative_Humidity", 
        "Mean_Mixed_Layer", 
        "Wind_Direction", 
        "Wind_Speed", 
        "Potential_Temp",
        "Equivalent_Potential_Temp",
        "Virtual_Potential_Temp"]

In [307]:
new_data = pd.DataFrame(data, columns=cols)

# Convert DataFrame to numeric values
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
for col in cols[2:]:
    try:
        new_data[col] = pd.to_numeric(new_data[col])
    except:
        print(col)
    
# Convert wind speed from Knots to KM/H
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
new_data['Wind_Speed'] = new_data['Wind_Speed'].multiply(1.852)
new_data.head(10)

,Date,Station,Pressure,Height,Temp,DewPoint,Relative_Humidity,Mean_Mixed_Layer,Wind_Direction,Wind_Speed,Potential_Temp,Equivalent_Potential_Temp,Virtual_Potential_Temp
0,2023-06-22,Quillayute,700,3130,-1.9,-6.9,69,3.27,15,27.780,300.4,310.5,300.9
1,2023-06-22,Quillayute,850,1551,8.6,-1.4,49,4.08,320,14.816,295.1,307.4,295.9
2,2023-06-22,Vernon,700,3108,-3.5,-5.7,85,3.59,275,12.964,298.6,309.6,299.2
3,2023-06-22,Vernon,850,1539,8.8,-0.2,53,4.46,235,11.112,295.4,308.7,296.1
4,2023-06-22,Port Hardy,700,3127,-1.1,-16.1,31,1.56,325,14.816,301.2,306.3,301.5
5,2023-06-22,Port Hardy,850,1550,9.4,-2.6,43,3.73,295,14.816,296.0,307.3,296.6
6,2023-06-23,Quillayute,700,3088,0.6,-12.4,37,2.12,55,18.520,303.1,309.9,303.5
7,2023-06-23,Quillayute,850,1501,11.4,-1.6,40,4.02,80,11.112,298.1,310.4,298.8
8,2023-06-23,Vernon,700,3103,-1.1,-7.1,64,3.22,165,11.112,301.2,311.3,301.8
9,2023-06-23,Vernon,850,1515,12.8,-0.2,41,4.46,270,11.112,299.5,313.2,300.4


In [308]:
new_data.tail()

,Date,Station,Pressure,Height,Temp,DewPoint,Relative_Humidity,Mean_Mixed_Layer,Wind_Direction,Wind_Speed,Potential_Temp,Equivalent_Potential_Temp,Virtual_Potential_Temp
37,2023-06-28,Quillayute,850,1498,15.0,-14.0,12,1.53,310,9.260,301.9,306.9,302.1
38,2023-06-28,Vernon,700,3113,6.0,-2.0,56,4.74,60,14.816,309.1,324.1,310.0
39,2023-06-28,Vernon,850,1479,20.8,7.8,43,7.87,355,16.668,307.9,332.2,309.4
40,2023-06-28,Port Hardy,700,3106,3.2,-7.8,44,3.05,225,11.112,306.0,315.8,306.6
41,2023-06-28,Port Hardy,850,1505,12.4,5.4,62,6.66,325,20.372,299.1,319.1,300.3


## Combine New and Old Data

In [309]:
complete_data = pd.concat([weather_data, new_data])
complete_data = complete_data.sort_values(by=['Date'])
#complete_data = complete_data.drop('Unnamed: 0', axis=1)
complete_data.head(20)

,Date,Station,Pressure,Height,Temp,DewPoint,Relative_Humidity,Mean_Mixed_Layer,Wind_Direction,Wind_Speed,Potential_Temp,Equivalent_Potential_Temp,Virtual_Potential_Temp
0,2020-01-01,Quillayute,700,2969.0,-0.9,-0.9,100.0,5.14,255.0,100.008,301.5,317.2,302.4
1,2020-01-01,Quillayute,850,1393.0,6.2,6.2,100.0,7.04,260.0,81.488,292.6,313.1,293.9
2,2020-01-01,Vernon,700,2880.0,-5.5,-7.0,89.0,3.25,260.0,94.452,296.4,306.3,296.9
3,2020-01-01,Vernon,850,1357.0,-3.9,-5.8,87.0,2.93,170.0,18.520,282.1,290.5,282.6
4,2020-01-01,Port Hardy,700,2856.0,-4.3,-5.8,89.0,3.56,250.0,87.044,297.7,308.6,298.3
5,2020-01-01,Port Hardy,850,1301.0,3.4,1.9,90.0,5.19,250.0,59.264,289.7,304.8,290.6
6,2020-01-02,Quillayute,700,2946.0,-10.7,-23.7,34.0,0.81,260.0,50.004,290.6,293.2,290.8
7,2020-01-02,Quillayute,850,1426.0,-1.3,-1.6,98.0,4.02,280.0,42.596,284.8,296.4,285.5
8,2020-01-02,Vernon,700,2882.0,-9.5,-12.3,80.0,2.14,305.0,31.484,291.9,298.5,292.3
9,2020-01-02,Vernon,850,1351.0,2.2,-4.8,60.0,3.16,245.0,33.336,288.4,297.8,289.0


In [310]:
complete_data.tail(20)

,Date,Station,Pressure,Height,Temp,DewPoint,Relative_Humidity,Mean_Mixed_Layer,Wind_Direction,Wind_Speed,Potential_Temp,Equivalent_Potential_Temp,Virtual_Potential_Temp
18,2023-06-25,Quillayute,700,3123.0,1.2,-16.8,25.0,1.47,355.0,3.704,303.8,308.6,304.1
20,2023-06-25,Vernon,700,3122.0,-0.1,-0.9,94.0,5.14,220.0,29.632,302.3,318.1,303.3
28,2023-06-26,Port Hardy,700,3142.0,2.4,-6.6,51.0,3.35,0.0,12.964,305.1,315.7,305.7
27,2023-06-26,Vernon,850,1529.0,14.4,7.4,63.0,7.65,145.0,1.852,301.2,324.2,302.6
29,2023-06-26,Port Hardy,850,1544.0,12.0,-9.0,22.0,2.29,350.0,3.704,298.7,305.9,299.1
25,2023-06-26,Quillayute,850,1541.0,12.8,-8.2,22.0,2.43,290.0,3.704,299.5,307.2,300.0
24,2023-06-26,Quillayute,700,3136.0,1.8,-22.2,15.0,0.93,325.0,9.260,304.4,307.6,304.6
26,2023-06-26,Vernon,700,3131.0,1.8,-1.9,76.0,4.78,355.0,7.408,304.4,319.3,305.3
30,2023-06-27,Quillayute,700,3122.0,2.0,-1.6,77.0,4.89,20.0,16.668,304.7,319.8,305.6
31,2023-06-27,Quillayute,850,1520.0,13.0,4.0,54.0,6.03,310.0,11.112,299.8,317.9,300.8


## Additional Cleaning on ALL Data
---

In [319]:
# Remove values that are exceptionally high or low
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
for col in cols[2:]:
    q_low = complete_data[col].quantile(0.01)
    q_hi = complete_data[col].quantile(0.99)
    
    complete_clean_data = complete_data[(complete_data[col] < q_hi) & (complete_data[col] > q_low)]

## Export Clean Data to CSV
---

In [311]:
# Save backup before overwriting weather.csv
filepath = Path('./data/backups/weather-{}.csv'.format(datetime.today().date()))
filepath.parent.mkdir(parents=True, exist_ok=True) 
weather_data.to_csv(filepath, index=False)

In [312]:
# Overwrite existing weather.csv with most recent data
filepath = Path('./data/weather.csv')
filepath.parent.mkdir(parents=True, exist_ok=True) 
complete_data.to_csv(filepath, index=False)

In [320]:
# Overwrite existing weather.csv with most recent data
filepath = Path('./data/weather_clean.csv')
filepath.parent.mkdir(parents=True, exist_ok=True) 
complete_clean_data.to_csv(filepath, index=False)